## 1. Load Dataset dan Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

In [ ]:
# Load dataset
try:
    df = pd.read_csv('https://github.com/erlanggadewasakti/Prinsip-Sains-Data/releases/download/prod/sa-psd-dataset.csv')
    print(f"Dataset loaded successfully! Shape: {df.shape}")
    display(df.head())
except Exception as e:
    print(f"Error loading dataset: {e}")

In [ ]:
# Preprocessing
df['sentiment'] = df['output'].str.replace(r'^[A-E]:\s*', '', regex=True)
df['cleaned_input'] = df['input'].str.lower()

# Map sentiment categories
df['sentiment'] = df['sentiment'].map({
    'very positive': 'positive',
    'very negative': 'negative',
    'positive': 'positive',
    'negative': 'negative',
    'neutral': 'neutral'
})

# Calculate text features
df['char_length'] = df['cleaned_input'].str.len()
df['word_count'] = df['cleaned_input'].str.split().str.len()
df['avg_word_length'] = df['char_length'] / df['word_count']
df['avg_word_length'] = df['avg_word_length'].fillna(0)

# Encode sentiment for correlation
sentiment_encoding = {'positive': 1, 'neutral': 0, 'negative': -1}
df['sentiment_encoded'] = df['sentiment'].map(sentiment_encoding)

print("Preprocessing completed!")
print(f"\nSentiment distribution:")
print(df['sentiment'].value_counts())

---
## 2. Visualisasi Distribusi Data

### 2.1 Distribusi Sentimen (Interactive)

In [ ]:
# Interactive Sentiment Distribution
sentiment_counts = df['sentiment'].value_counts().reset_index()
sentiment_counts.columns = ['sentiment', 'count']
sentiment_counts['percentage'] = (sentiment_counts['count'] / sentiment_counts['count'].sum() * 100).round(2)

# Color mapping
color_map = {'positive': '#2ecc71', 'negative': '#e74c3c', 'neutral': '#3498db'}

fig = px.bar(sentiment_counts,
             x='sentiment',
             y='count',
             color='sentiment',
             color_discrete_map=color_map,
             text='count',
             title='<b>Distribusi Sentimen dalam Dataset</b>',
             labels={'sentiment': 'Sentimen', 'count': 'Jumlah Data'},
             hover_data={'percentage': ':.2f'})

fig.update_traces(texttemplate='%{text}<br>(%{customdata[0]}%)',
                  textposition='outside',
                  customdata=sentiment_counts[['percentage']])

fig.update_layout(height=500,
                  showlegend=False,
                  title_font_size=18,
                  title_x=0.5,
                  xaxis_title_font_size=14,
                  yaxis_title_font_size=14)

fig.show()

In [ ]:
# Pie Chart Interactive
fig = px.pie(sentiment_counts,
             values='count',
             names='sentiment',
             color='sentiment',
             color_discrete_map=color_map,
             title='<b>Proporsi Sentimen</b>',
             hole=0.4)

fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  marker=dict(line=dict(color='white', width=2)))

fig.update_layout(height=500,
                  title_font_size=18,
                  title_x=0.5)

fig.show()

### 2.2 Distribusi Fitur Teks (Character Length, Word Count, Average Word Length)

In [ ]:
# Interactive Histogram untuk semua fitur teks
text_features = ['char_length', 'word_count', 'avg_word_length']
feature_labels = {
    'char_length': 'Panjang Karakter',
    'word_count': 'Jumlah Kata',
    'avg_word_length': 'Rata-rata Panjang Kata'
}

for feature in text_features:
    fig = px.histogram(df,
                       x=feature,
                       color='sentiment',
                       color_discrete_map=color_map,
                       marginal='box',
                       nbins=50,
                       title=f'<b>Distribusi {feature_labels[feature]} berdasarkan Sentimen</b>',
                       labels={feature: feature_labels[feature], 'sentiment': 'Sentimen'},
                       opacity=0.7)

    fig.update_layout(height=500,
                      title_font_size=16,
                      title_x=0.5,
                      barmode='overlay')

    fig.show()

In [ ]:
# Box Plot Interactive untuk perbandingan distribusi
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Panjang Karakter', 'Jumlah Kata', 'Rata-rata Panjang Kata')
)

for idx, feature in enumerate(text_features, 1):
    for sentiment in df['sentiment'].unique():
        sentiment_data = df[df['sentiment'] == sentiment][feature]
        fig.add_trace(
            go.Box(y=sentiment_data,
                   name=sentiment,
                   marker_color=color_map[sentiment],
                   showlegend=(idx == 1)),
            row=1, col=idx
        )

fig.update_layout(height=500,
                  title_text='<b>Perbandingan Distribusi Fitur Teks Antar Sentimen</b>',
                  title_font_size=18,
                  title_x=0.5,
                  showlegend=True)

fig.show()

### 2.3 Violin Plot - Distribusi Detail

In [ ]:
# Violin Plot untuk melihat distribusi lebih detail
for feature in text_features:
    fig = px.violin(df,
                    x='sentiment',
                    y=feature,
                    color='sentiment',
                    color_discrete_map=color_map,
                    box=True,
                    points='outliers',
                    title=f'<b>Distribusi {feature_labels[feature]} per Sentimen (Violin Plot)</b>',
                    labels={feature: feature_labels[feature], 'sentiment': 'Sentimen'})

    fig.update_layout(height=500,
                      title_font_size=16,
                      title_x=0.5,
                      showlegend=False)

    fig.show()

---
## 3. Visualisasi Hubungan Antar Variabel

### 3.1 Scatter Plot Matrix - Hubungan Antar Fitur

In [ ]:
# Scatter Matrix Interactive
fig = px.scatter_matrix(df,
                        dimensions=['char_length', 'word_count', 'avg_word_length'],
                        color='sentiment',
                        color_discrete_map=color_map,
                        title='<b>Scatter Matrix - Hubungan Antar Fitur Teks</b>',
                        labels={
                            'char_length': 'Panjang Karakter',
                            'word_count': 'Jumlah Kata',
                            'avg_word_length': 'Avg Word Length'
                        },
                        opacity=0.6,
                        height=800)

fig.update_traces(diagonal_visible=False,
                  showupperhalf=False,
                  marker=dict(size=3))

fig.update_layout(title_font_size=18,
                  title_x=0.5)

fig.show()

### 3.2 Scatter Plot Individual - Analisis Detail

In [ ]:
# Character Length vs Word Count
fig = px.scatter(df,
                 x='char_length',
                 y='word_count',
                 color='sentiment',
                 color_discrete_map=color_map,
                 title='<b>Hubungan Panjang Karakter vs Jumlah Kata</b>',
                 labels={'char_length': 'Panjang Karakter', 'word_count': 'Jumlah Kata'},
                 opacity=0.6,
                 trendline='ols',
                 hover_data=['sentiment', 'avg_word_length'])

fig.update_layout(height=550,
                  title_font_size=16,
                  title_x=0.5)

fig.show()

In [ ]:
# Word Count vs Average Word Length
fig = px.scatter(df,
                 x='word_count',
                 y='avg_word_length',
                 color='sentiment',
                 color_discrete_map=color_map,
                 title='<b>Hubungan Jumlah Kata vs Rata-rata Panjang Kata</b>',
                 labels={'word_count': 'Jumlah Kata', 'avg_word_length': 'Rata-rata Panjang Kata'},
                 opacity=0.6,
                 trendline='ols',
                 hover_data=['sentiment', 'char_length'])

fig.update_layout(height=550,
                  title_font_size=16,
                  title_x=0.5)

fig.show()

### 3.3 Correlation Heatmap Interactive

In [ ]:
# Correlation Matrix dengan Sentiment
correlation_features = ['char_length', 'word_count', 'avg_word_length', 'sentiment_encoded']
corr_matrix = df[correlation_features].corr()

# Rename untuk tampilan yang lebih baik
display_names = {
    'char_length': 'Panjang Karakter',
    'word_count': 'Jumlah Kata',
    'avg_word_length': 'Avg Word Length',
    'sentiment_encoded': 'Sentimen'
}

corr_matrix_renamed = corr_matrix.rename(columns=display_names, index=display_names)

fig = px.imshow(corr_matrix_renamed,
                text_auto='.3f',
                color_continuous_scale='RdBu_r',
                aspect='auto',
                title='<b>Heatmap Korelasi - Fitur Teks dan Sentimen</b>',
                zmin=-1, zmax=1)

fig.update_layout(height=550,
                  title_font_size=16,
                  title_x=0.5)

fig.show()

In [ ]:
# Correlation Heatmap per Sentiment
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Positive', 'Neutral', 'Negative'),
    horizontal_spacing=0.15
)

sentiments = ['positive', 'neutral', 'negative']
for idx, sentiment in enumerate(sentiments, 1):
    sentiment_df = df[df['sentiment'] == sentiment]
    corr = sentiment_df[text_features].corr()

    fig.add_trace(
        go.Heatmap(z=corr.values,
                   x=['Char Len', 'Word Cnt', 'Avg Word'],
                   y=['Char Len', 'Word Cnt', 'Avg Word'],
                   colorscale='RdBu_r',
                   zmid=0,
                   zmin=-1, zmax=1,
                   text=corr.values,
                   texttemplate='%{text:.2f}',
                   showscale=(idx == 3)),
        row=1, col=idx
    )

fig.update_layout(height=500,
                  title_text='<b>Korelasi Fitur Teks per Sentimen</b>',
                  title_font_size=18,
                  title_x=0.5)

fig.show()

### 3.4 Parallel Coordinates - Hubungan Multi-Dimensi

In [ ]:
# Parallel Coordinates Plot
fig = px.parallel_coordinates(
    df,
    dimensions=['char_length', 'word_count', 'avg_word_length'],
    color='sentiment_encoded',
    color_continuous_scale=[(0, '#e74c3c'), (0.5, '#3498db'), (1, '#2ecc71')],
    labels={
        'char_length': 'Panjang<br>Karakter',
        'word_count': 'Jumlah<br>Kata',
        'avg_word_length': 'Avg Word<br>Length',
        'sentiment_encoded': 'Sentimen'
    },
    title='<b>Parallel Coordinates - Hubungan Multi-Dimensi Fitur Teks</b>'
)

fig.update_layout(height=600,
                  title_font_size=16,
                  title_x=0.5)

fig.show()

---
## 4. Dashboard Interaktif Komprehensif

### 4.1 Dashboard Overview - Statistik Utama

In [ ]:
# Dashboard dengan multiple subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Prepare data
sentiment_stats = df.groupby('sentiment').agg({
    'char_length': ['mean', 'median'],
    'word_count': ['mean', 'median'],
    'avg_word_length': ['mean', 'median']
}).reset_index()

sentiment_stats.columns = ['sentiment', 'char_mean', 'char_median', 'word_mean', 'word_median', 'avg_mean', 'avg_median']

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribusi Sentimen',
                    'Mean Panjang Karakter per Sentimen',
                    'Mean Jumlah Kata per Sentimen',
                    'Mean Rata-rata Panjang Kata per Sentimen'),
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# 1. Pie chart - Sentiment Distribution
sentiment_counts = df['sentiment'].value_counts()
colors = [color_map[s] for s in sentiment_counts.index]
fig.add_trace(
    go.Pie(labels=sentiment_counts.index,
           values=sentiment_counts.values,
           marker=dict(colors=colors),
           hole=0.3),
    row=1, col=1
)

# 2. Bar chart - Character Length
for sentiment in sentiment_stats['sentiment']:
    row_data = sentiment_stats[sentiment_stats['sentiment'] == sentiment]
    fig.add_trace(
        go.Bar(x=[sentiment],
               y=row_data['char_mean'],
               name=sentiment,
               marker_color=color_map[sentiment],
               showlegend=False),
        row=1, col=2
    )

# 3. Bar chart - Word Count
for sentiment in sentiment_stats['sentiment']:
    row_data = sentiment_stats[sentiment_stats['sentiment'] == sentiment]
    fig.add_trace(
        go.Bar(x=[sentiment],
               y=row_data['word_mean'],
               name=sentiment,
               marker_color=color_map[sentiment],
               showlegend=False),
        row=2, col=1
    )

# 4. Bar chart - Average Word Length
for sentiment in sentiment_stats['sentiment']:
    row_data = sentiment_stats[sentiment_stats['sentiment'] == sentiment]
    fig.add_trace(
        go.Bar(x=[sentiment],
               y=row_data['avg_mean'],
               name=sentiment,
               marker_color=color_map[sentiment],
               showlegend=False),
        row=2, col=2
    )

fig.update_layout(height=800,
                  title_text='<b>Dashboard Overview - Sentiment Analysis Statistics</b>',
                  title_font_size=20,
                  title_x=0.5,
                  showlegend=False)

fig.show()

### 4.2 Dashboard Distribusi - Histogram & Box Plot

In [ ]:
# Dashboard dengan histogram dan boxplot
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Histogram - Panjang Karakter', 'Box Plot - Panjang Karakter',
                    'Histogram - Jumlah Kata', 'Box Plot - Jumlah Kata',
                    'Histogram - Avg Word Length', 'Box Plot - Avg Word Length'),
    column_widths=[0.6, 0.4],
    horizontal_spacing=0.1,
    vertical_spacing=0.1
)

features = ['char_length', 'word_count', 'avg_word_length']
sentiments_list = df['sentiment'].unique()

for idx, feature in enumerate(features, 1):
    # Histogram
    for sentiment in sentiments_list:
        sentiment_data = df[df['sentiment'] == sentiment][feature]
        fig.add_trace(
            go.Histogram(x=sentiment_data,
                        name=sentiment,
                        marker_color=color_map[sentiment],
                        opacity=0.6,
                        showlegend=(idx == 1)),
            row=idx, col=1
        )

    # Box Plot
    for sentiment in sentiments_list:
        sentiment_data = df[df['sentiment'] == sentiment][feature]
        fig.add_trace(
            go.Box(y=sentiment_data,
                   name=sentiment,
                   marker_color=color_map[sentiment],
                   showlegend=False),
            row=idx, col=2
        )

fig.update_layout(height=1000,
                  title_text='<b>Dashboard Distribusi - Analisis Fitur Teks per Sentimen</b>',
                  title_font_size=20,
                  title_x=0.5,
                  barmode='overlay')

fig.show()

### 4.3 Dashboard Perbandingan - Mean vs Median

In [ ]:
# Dashboard perbandingan Mean vs Median
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Panjang Karakter', 'Jumlah Kata', 'Avg Word Length')
)

features_comparison = {
    'char_length': ['char_mean', 'char_median'],
    'word_count': ['word_mean', 'word_median'],
    'avg_word_length': ['avg_mean', 'avg_median']
}

for idx, (feature, cols) in enumerate(features_comparison.items(), 1):
    sentiments_order = ['positive', 'neutral', 'negative']

    for sentiment in sentiments_order:
        row_data = sentiment_stats[sentiment_stats['sentiment'] == sentiment]

        # Mean
        fig.add_trace(
            go.Bar(x=[f"{sentiment}<br>Mean"],
                   y=row_data[cols[0]],
                   name='Mean',
                   marker_color=color_map[sentiment],
                   opacity=0.8,
                   showlegend=(idx == 1 and sentiment == 'positive')),
            row=1, col=idx
        )

        # Median
        fig.add_trace(
            go.Bar(x=[f"{sentiment}<br>Median"],
                   y=row_data[cols[1]],
                   name='Median',
                   marker_color=color_map[sentiment],
                   opacity=0.5,
                   showlegend=(idx == 1 and sentiment == 'positive')),
            row=1, col=idx
        )

fig.update_layout(height=500,
                  title_text='<b>Dashboard Perbandingan Mean vs Median per Sentimen</b>',
                  title_font_size=18,
                  title_x=0.5,
                  barmode='group')

fig.show()

### 4.4 Dashboard Scatter - Hubungan Antar Variabel

In [ ]:
# Dashboard dengan scatter plots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Char Length vs Word Count',
                    'Char Length vs Avg Word Length',
                    'Word Count vs Avg Word Length',
                    'Distribusi 3D'),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter3d'}]]
)

# Scatter plot 1: char_length vs word_count
for sentiment in sentiments_list:
    sentiment_df = df[df['sentiment'] == sentiment]
    fig.add_trace(
        go.Scatter(x=sentiment_df['char_length'],
                   y=sentiment_df['word_count'],
                   mode='markers',
                   name=sentiment,
                   marker=dict(color=color_map[sentiment], size=4, opacity=0.5),
                   showlegend=True),
        row=1, col=1
    )

# Scatter plot 2: char_length vs avg_word_length
for sentiment in sentiments_list:
    sentiment_df = df[df['sentiment'] == sentiment]
    fig.add_trace(
        go.Scatter(x=sentiment_df['char_length'],
                   y=sentiment_df['avg_word_length'],
                   mode='markers',
                   name=sentiment,
                   marker=dict(color=color_map[sentiment], size=4, opacity=0.5),
                   showlegend=False),
        row=1, col=2
    )

# Scatter plot 3: word_count vs avg_word_length
for sentiment in sentiments_list:
    sentiment_df = df[df['sentiment'] == sentiment]
    fig.add_trace(
        go.Scatter(x=sentiment_df['word_count'],
                   y=sentiment_df['avg_word_length'],
                   mode='markers',
                   name=sentiment,
                   marker=dict(color=color_map[sentiment], size=4, opacity=0.5),
                   showlegend=False),
        row=2, col=1
    )

# 3D Scatter plot
for sentiment in sentiments_list:
    sentiment_df = df[df['sentiment'] == sentiment]
    fig.add_trace(
        go.Scatter3d(x=sentiment_df['char_length'],
                     y=sentiment_df['word_count'],
                     z=sentiment_df['avg_word_length'],
                     mode='markers',
                     name=sentiment,
                     marker=dict(color=color_map[sentiment], size=2, opacity=0.5),
                     showlegend=False),
        row=2, col=2
    )

# Update axes labels
fig.update_xaxes(title_text="Panjang Karakter", row=1, col=1)
fig.update_yaxes(title_text="Jumlah Kata", row=1, col=1)

fig.update_xaxes(title_text="Panjang Karakter", row=1, col=2)
fig.update_yaxes(title_text="Avg Word Length", row=1, col=2)

fig.update_xaxes(title_text="Jumlah Kata", row=2, col=1)
fig.update_yaxes(title_text="Avg Word Length", row=2, col=1)

fig.update_layout(height=900,
                  title_text='<b>Dashboard Scatter - Analisis Hubungan Multi-Variabel</b>',
                  title_font_size=20,
                  title_x=0.5)

fig.show()

---
## 5. Visualisasi Statistik Lanjutan

### 5.1 Density Plot - Distribusi Probabilitas

In [ ]:
# Density plots untuk setiap fitur
for feature in text_features:
    fig = go.Figure()

    for sentiment in sentiments_list:
        sentiment_data = df[df['sentiment'] == sentiment][feature]

        # Create histogram with density
        fig.add_trace(go.Histogram(
            x=sentiment_data,
            name=sentiment,
            opacity=0.6,
            marker_color=color_map[sentiment],
            histnorm='probability density',
            nbinsx=50
        ))

    fig.update_layout(
        title=f'<b>Density Plot - {feature_labels[feature]}</b>',
        xaxis_title=feature_labels[feature],
        yaxis_title='Density',
        barmode='overlay',
        height=500,
        title_font_size=16,
        title_x=0.5
    )

    fig.show()

### 5.2 Cumulative Distribution Function (CDF)

In [ ]:
# CDF plots
for feature in text_features:
    fig = go.Figure()

    for sentiment in sentiments_list:
        sentiment_data = df[df['sentiment'] == sentiment][feature].sort_values()
        cumulative = np.arange(1, len(sentiment_data) + 1) / len(sentiment_data)

        fig.add_trace(go.Scatter(
            x=sentiment_data,
            y=cumulative,
            name=sentiment,
            line=dict(color=color_map[sentiment], width=3),
            mode='lines'
        ))

    fig.update_layout(
        title=f'<b>Cumulative Distribution Function - {feature_labels[feature]}</b>',
        xaxis_title=feature_labels[feature],
        yaxis_title='Cumulative Probability',
        height=500,
        title_font_size=16,
        title_x=0.5
    )

    fig.show()

### 5.3 Summary Statistics Table

In [ ]:
# Create summary statistics table
summary_list = []

for sentiment in sentiments_list:
    sentiment_df = df[df['sentiment'] == sentiment]

    for feature in text_features:
        summary_list.append({
            'Sentimen': sentiment,
            'Fitur': feature_labels[feature],
            'Count': sentiment_df[feature].count(),
            'Mean': sentiment_df[feature].mean(),
            'Median': sentiment_df[feature].median(),
            'Std': sentiment_df[feature].std(),
            'Min': sentiment_df[feature].min(),
            'Max': sentiment_df[feature].max(),
            'Q25': sentiment_df[feature].quantile(0.25),
            'Q75': sentiment_df[feature].quantile(0.75)
        })

summary_df = pd.DataFrame(summary_list)

# Create interactive table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Sentimen</b>', '<b>Fitur</b>', '<b>Count</b>', '<b>Mean</b>', '<b>Median</b>',
                '<b>Std</b>', '<b>Min</b>', '<b>Max</b>', '<b>Q25</b>', '<b>Q75</b>'],
        fill_color='paleturquoise',
        align='center',
        font=dict(size=12, color='black')
    ),
    cells=dict(
        values=[summary_df[col] for col in summary_df.columns],
        fill_color='lavender',
        align='center',
        format=["", "", "", ".2f", ".2f", ".2f", ".2f", ".2f", ".2f", ".2f"],
        font=dict(size=11)
    ))
])

fig.update_layout(
    title='<b>Tabel Statistik Deskriptif - Fitur Teks per Sentimen</b>',
    title_font_size=18,
    title_x=0.5,
    height=600
)

fig.show()

---
## 6. Kesimpulan dan Insight

### Ringkasan Insight dari Visualisasi:

1. **Distribusi Sentimen:**
   - Dataset menunjukkan distribusi yang relatif seimbang/tidak seimbang antar ketiga kategori sentimen
   - Proporsi masing-masing sentimen dapat dilihat pada visualisasi pie chart dan bar chart

2. **Karakteristik Teks per Sentimen:**
   - **Panjang Karakter**: Analisis menunjukkan pola panjang teks yang berbeda antar sentimen
   - **Jumlah Kata**: Distribusi jumlah kata memberikan insight tentang verbositas setiap sentimen
   - **Rata-rata Panjang Kata**: Indikator kompleksitas vocabular yang digunakan

3. **Hubungan Antar Variabel:**
   - Korelasi positif kuat antara `char_length` dan `word_count` (expected)
   - `avg_word_length` relatif independen terhadap panjang teks
   - Scatter matrix menunjukkan pola clustering berdasarkan sentimen

4. **Outliers dan Anomali:**
   - Beberapa data point menunjukkan karakteristik ekstrem yang perlu investigasi lebih lanjut
   - Box plots membantu identifikasi outliers per kategori sentimen

5. **Rekomendasi untuk Modeling:**
   - Fitur teks menunjukkan perbedaan yang dapat dimanfaatkan untuk klasifikasi
   - Feature engineering dapat ditingkatkan dengan mempertimbangkan interaksi antar fitur
   - Perlu handling outliers untuk meningkatkan performa model

---

## 📊 Catatan Penggunaan Dashboard:

- **Interaktivitas**: Semua visualisasi Plotly mendukung zoom, pan, dan hover untuk detail
- **Export**: Klik icon kamera untuk download visualisasi sebagai PNG
- **Legend**: Klik legend untuk hide/show kategori tertentu
- **Responsive**: Dashboard otomatis menyesuaikan dengan ukuran layar

---

**Dibuat untuk Tugas 5 - Prinsip Sains Data**

**Topik**: Sentiment Analysis - Visualisasi dan Dashboard Interaktif